<a href="https://colab.research.google.com/github/parsa-abbasi/Sentiment-Analysis/blob/master/SentiPers/Classifier/BaseLine/GoogleColab/WithoutDataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hazm
!pip install stopwords_guilannlp

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from hazm import *
from stopwords_guilannlp import stopwords_output
import pandas as pd
import numpy as np

In [0]:
from google.colab import files
uploaded = files.upload()

In [31]:
x_train = pd.Series.from_csv('x_train.csv', sep='\t')
x_test = pd.Series.from_csv('x_test.csv', sep='\t')
y_train = pd.Series.from_csv('y_train.csv', sep='\t', header=0)
y_test = pd.Series.from_csv('y_test.csv', sep='\t', header=0)
x_train = x_train.iloc[1:, ]
x_test = x_test.iloc[1:, ]
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

x_train shape:  (5561,)
x_test shape:  (1854,)
y_train shape:  (5561,)
y_test shape:  (1854,)


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [0]:
# Make stop word set
stop_set = stopwords_output("Persian", "set")


# When building the vocabulary ignore terms that have a document frequency strictly lower than
# the given threshold. This value is also called cut-off in the literature.
min_df = 1


# Tokenize function used in Vectorizer
def tokenize(text):
    return word_tokenize(text)


In [33]:
# (Multinomial) Naive Bayes Model
text_clf = Pipeline([('vect', CountVectorizer(tokenizer=tokenize,
                                              analyzer='word', ngram_range=(1, 2), min_df=min_df, lowercase=False)),
                     ('tfidf', TfidfTransformer(sublinear_tf=True)),
                     ('clf', MultinomialNB())])
text_clf = text_clf.fit(x_train, y_train)
naive_score = text_clf.score(x_test, y_test)
print('Naive Bayes Model: ', naive_score)
predict = text_clf.predict(x_test)


Naive Bayes Model:  0.48759439050701187


In [34]:
# SGD (Stochastic Gradient Descent) Model
text_clf_sgd = Pipeline([('vect', CountVectorizer(tokenizer=tokenize,
                                                  analyzer='word', ngram_range=(1, 2), min_df=min_df, lowercase=False)),
                         ('tfidf', TfidfTransformer(sublinear_tf=True)),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                   alpha=1e-3, max_iter=5))])
text_clf_sgd = text_clf_sgd.fit(x_train, y_train)
sgd_score = text_clf_sgd.score(x_test, y_test)
print('SGD Model: ', sgd_score)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGD Model:  0.5938511326860841


In [35]:
# Linear Support Vector Machine Model
text_clf_linear_svc = Pipeline([('vect', CountVectorizer(tokenizer=tokenize,
                                                         analyzer='word', ngram_range=(1, 2),
                                                         min_df=min_df, lowercase=False)),
                                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                                ('clf-svm', LinearSVC(loss='hinge', penalty='l2',
                                                      max_iter=5))])

text_clf_linear_svc = text_clf_linear_svc.fit(x_train, y_train)
linear_svc_score = text_clf_linear_svc.score(x_test, y_test)
print('Linear SVC Model: ', linear_svc_score)


Linear SVC Model:  0.6456310679611651


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
